# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [199]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [208]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
r = requests.get(url)
pvl_RJ_json = r.json()
RJ_pvl = pd.DataFrame(pvl_RJ_json['items'])
RJ_pvl.head(n=3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13271,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001999/2008-12,2009-07-01T03:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.950000e+07,1,0,07/07/2009
1,21388,Estado,Rio de Janeiro,33,RJ,00000.000000/2024-66,Encaminhado à PGFN com manifestação técnica fa...,17944.000158/2016-06,2016-02-19T02:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,4.448111e+08,1,0,26/02/2016
2,5933,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000237/2012-85,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,1.000000e+08,1,0,10/07/2012


In [201]:
# 2) Seu código aqui
RJ_pvl['status'].value_counts()


Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [202]:
# 3) Seu código aqui
ano = str(RJ_pvl['data_status']).replace('/', ' ').split()
ano_lista = []
for c in ano:
    if len(c) == 4:
        ano_lista.append(c)
RJ_pvl['ano'] = ano_lista



In [203]:
#4) Seu código aqui
RJ_pvl['ano'].value_counts()

2012    11
2009     9
2013     7
2011     6
2016     4
2014     4
2002     4
2010     3
2008     2
2006     2
Name: ano, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [204]:
#1) Seu código aqui
def Api(UF = '', interessado = 'Estado'):
    import pandas as pd
    import requests
    
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=' + UF + '&tipo_interessado=' + interessado
    r = requests.get(url)
    data_json = r.json()
    df = pd.DataFrame(data_json['items'])
    return df


In [205]:
# 2) Seu código aqui
MG_pvl = Api('MG')
cond = MG_pvl['status'] == 'Arquivado por decurso de prazo'
MG_pvl[cond]['status'].value_counts()


Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [206]:
# 3) Seu código aqui
BA_pvl = Api('BA', 'Município')
cond = BA_pvl['status'] == 'Deferido'
BA_pvl[cond]['interessado'].value_counts()

Luís Eduardo Magalhães    14
Vitória da Conquista      11
Lauro de Freitas          10
Camaçari                  10
Barreiras                  9
                          ..
São Felipe                 1
Cravolândia                1
Ibititá                    1
Gavião                     1
Nazaré                     1
Name: interessado, Length: 175, dtype: int64

In [207]:
# 4) Seu código aqui
BA_pvl_estado = Api('BA')
BA_pvl_estado.to_csv('BA_pvl_estado.csv', sep=';')